# Hurdle 1 Code Walkthrough

## Introduction
All of the code discussed below resides under phase1-hurdles/hurdle1, available at:

https://github.com/SpectrumCollaborationChallenge/phase1-hurdles

Performer solutions for Hurdle 1 must be able to read complex samples from a TCP port, demodulate the samples, and decode the packets the samples contain. Then the solution must return decoded packets over another TCP port. In both cases, performer code will be a TCP client. 

The Hurdle 1 folder includes code that runs inside the Test Infrastructure container. It also includes an example GNU Radio script to read the samples from the Test Infrastructure, save them to disk, and return some number of random bytes back to the test infrastructure container. 

## The Build Script
build_hurdle_1_trusty.sh is a bash script that downloads a Ubuntu 14.04.5 64 bit LXC template and preconfigures it for use on Hurdle 1. It creates two containers, the Test Infrastructure container and the Solution container. The Solution container is a simple clone of the Test Infrastructure container. These containers will be stored in phase1-hurdles/hurdle1/lxc. Each container requires several GB of disk space, so ensure there is sufficient hard disk space before proceeding.


## Example Solution Code

## hurdle_1_solution_stub.grc and hurdle_1_solution_stub.py
hurdle_1_solution_stub.grc is a GNU Radio flowgraph that runs in the Solution container that creates a TCP client to listen to samples generated by the Test Infrastructure container. It also generates a random series of bytes and sends them back to the Test Infrastructure container. It is not an actual solution and should be replaced by Open Track teams. 

Open track teams may use this file to record samples from the Test Infrastructure for offline processing during early development. The samples will be recorded in the standard GNU Radio Complex format. Users of MATLAB or Octave may process these files using utility scripts such as read_complex_binary.m. Teams may find this script in the GNU Radio source code under gnuradio/gr-utils/octave/read_complex_binary.m or at:

http://gnuradio.org/redmine/projects/gnuradio/repository/revisions/master/entry/gr-utils/octave/read_complex_binary.m

Python users may read these files with scipy as

In [1]:
import scipy

def read_samples_scipy(filename, blocklen):
    
    iq_samples = scipy.fromfile(filename, 
                                dtype=scipy.complex64, 
                                count=blocklen)
    return iq_samples

or with numpy as 

In [2]:
import numpy

def read_samples_numpy(filename, blocklen):
    
    iq_samples = numpy.fromfile(filename,
                                dtype=numpy.complex64,
                                count=blocklen)
    
    return iq_samples

## Test Infrastructure Code

### gr-hurdle1
The gr-hurdle1 folder contains a GNU Radio Out-of-Tree module, see:

http://gnuradio.org/redmine/projects/gnuradio/wiki/OutOfTreeModules

These Out-of-Tree modules are used to add capability to GNU Radio that isn't necessarily expected to get merged into the mainline GNU Radio source code. In this case, we've developed some custom GNU Radio blocks to support Hurdle 1, as explained in the two sections below:

### gr-hurdle1:random_packet_source

The first block of interest from the Test Infrastructure Code is random_packet_source. This block generates some number of payload bits spread across packets of varying sizes. This block also picks a random packet-to-packet spacing between packets. These random packet-to-packet spacing values are included in an outer frame header, which is purely an implementation detail and not included in the modulated packet data.

The packet header fields visible to the performers are only those listed in the Hurdle 1 Problem Statement: the preamble, sync word, the four copies of the packet length, and the four copies of the packet counter. The CRC field generated by gen_packets.py is an implementation detail and should not be relied upon by Open Track Teams, as it is subject to change without notice. 

### gr-hurdle1:hurdle1_traffic_parser
This block is implemented in Python and is the counterpart to gen_packets.py. Hurdle1_traffic_parser reads the outer frame header generated by gen_packets.py and converts the header fields into metadata associated with the first byte of the packet. Hurdle1_traffic_parser then outputs the data stream, still in byte format, with the outer frame header fields removed. The metadata from the frame header is passed along as a GNU Radio stream tag associated with the first byte of the packet. 

### gr-hurdle1:hurdle1_zero_pad
This block reads the metadata tags generated by hurdle1_traffic_parser and adds a series of zeros to the start of the packet, as dictated by the zero-padding field in the metadata tag. For Hurdle 1, this block will operate on the complex samples from the output of the QPSK modulator. 

## hurdle_1.grc and hurdle_1.py
hurdle_1.grc is a GNU Radio flowgraph that processes the packets from gen_packets.py and converts them into a stream of QPSK symbols embedded in complex baseband I/Q samples with variable amounts of delay from packet to packet, and makes the samples available via a TCP server listening on a predefined port. This flowgraph also runs a TCP server listening for packets sent by the Solution container on a predefined port. It currently is set up to write these packets out to disk. The GRC file dictates the interconnection of various GNU Radio blocks, both from the standard GNU Radio source as well as from the gr-hurdle1 module. 

The gnuradio-companion application is used to automatically generate executable python from this GRC file. This file is included as hurdle_1.py.

## calc_ber.py
This Python script operates on the file saved off by hurdle_1_rx.py to calculate the final Bit Error Rate (BER) of the Open Track Team's solution and generate an overall hurdle Pass/Fail. 


## Items To Be Updated
All updates will be made available on:

https://github.com/SpectrumCollaborationChallenge/phase1-hurdles

Open Track teams will also be notified of updates via email. 